In [1]:
import numpy as np
import random

In [2]:
trial1=100
trial2=500
trial3=1000

In [3]:
4*4*4

64

In [ ]:
#x_board or o_board is the different grib positions
#x_score and o_score is the utility values 

In [ ]:
#playes the game a number of times with a learning parameter
def play_game(num,learn_rate):
    x_board,x_score,o_board,o_score=list()
    for i in range(0,num):
        grid,x_board,x_score,o_board,o_score=setupGame(print_=False,move_type="learn",x_board,x_score,o_board,o_score,learn_rate)
    

In [23]:
#pathX or pathO is a gird with each location has a number where X and O is played
def update_path(loc,current_path,move_num):
    current_path[loc[0],loc[1],loc[2]]=move_num
    return current_path
    

In [ ]:
#the first learning function that is called
def first_update_learn(current_path,grid,status,board,score,learn_rate):
    if status="winner":
        #location of the last move
        i,j,k=np.where(current_path == np.max(current_path))
        #checks if it is previously had an example
        index=np.where((board == grid).all(axis=1))
        #if not
        if len(index)==0:
            #add the grid to the boards
            board.append(grid)
            #add the scoring board
            cur_score=np.zeros(shape=(4,4,4),dtype=np.dtype('i4'))
            cur_score[i,j,k]=1
            score.append(cur_score)
        else:
            score[index,i,j,k]=score[index,i,j,k]+1
        return update_learn(current_path,grid,board,score,learn_rate,index,[i,j,k])
    #same thing as winner but instead of adding one to the score, it takes away from it 
    elif status="loser":
        i,j,k=np.where(current_path == np.max(current_path))
        index=np.where((board == grid).all(axis=1))
        if len(index)==0:
            board.append(grid)
            cur_score=np.zeros(shape=(4,4,4),dtype=np.dtype('i4'))
            cur_score[i,j,k]=-1
            score.append(cur_score)
        else:
            score[index,i,j,k]=score[index,i,j,k]-1
        return update_learn(current_path,grid,board,score,learn_rate,index,[i,j,k])
    else:
        i,j,k=np.where(current_path == np.max(current_path))
        index=np.where((board == grid).all(axis=1))
        if len(index)==0:
            board.append(grid)
            cur_score=np.zeros(shape=(4,4,4),dtype=np.dtype('i4'))
            cur_score[i,j,k]=0
            score.append(cur_score)
        return update_learn(current_path,grid,board,score,learn_rate,index,[i,j,k])

In [ ]:
def update_learn(current_path,grid,board,score,learn_rate,index):
    #not the starting update learn function but a recursive one
    #find latest step
    i,j,k=np.where(current_path == np.max(current_path))
    #set that spot empty to go back in time
    grid[i,j,k]=""
    current_path[i,j,k]=0
    #go to next latest step
    x,y,z=np.where(current_path == np.max(current_path))
    #check if the current board is in the playing boards
    new_index=np.where((board == grid).all(axis=1))
    if len(new_index)==0:
            board.append(grid)
            cur_score=np.zeros(shape=(4,4,4),dtype=np.dtype('i4'))
            cur_score[x,y,z]=score[index,i,j,k]*learn_rate
            score.append(cur_score)
    else:
        score[new_index,x,y,z]=score[index,i,j,k]*learn_rate+score[new_index,x,y,z]
    if np.max(current_path)==1:
        return board,score
    else:
        update_learn(current_path,grid,board,score,learn_rate,new_index)

In [27]:
def setupGame(print_=False,move_type="learn",x_board,x_score,o_board,o_score,learn_rate):
    #setup grid
    grid=np.zeros(shape=(4,4,4),dtype=np.dtype('U25'))
    #there can be max 64 plays
    x_move=1
    o_move=1
    x_path=np.zeros(shape=(4,4,4),dtype=np.dtype('i4'))
    o_path=np.zeros(shape=(4,4,4),dtype=np.dtype('i4'))
    for i in range(0,64):
        # x moves
        if move_type=="rand":
            grid,loc=rand_move(grid,"X")
        elif move_type=="learn":
            grid,loc=learn_move(grid,"X",x_board,x_score)
        #check if game is over
        x_path=update_path(loc,x_path,x_move)
        x_move=x_move+1
        solved,direction=checkWinner(grid,"X",loc)
        if solved:
            print("X wins")
            if print_:
                print("loc= ")
                print(loc)
                print("direction= ")
                print(direction)
            x_board,x_score=first_update_learn(x_path,grid.copy(),"winner",x_board,x_score,learn_rate)
            o_board,o_score=first_update_learn(o_path,grid.copy(),"loser",o_board,o_score,learn_rate)
            return grid,x_board,x_score,o_board,o_score
        # o moves
        if move_type=="rand":
            grid,loc=rand_move(grid,"O")
        elif move_type=="learn":
            grid,loc=learn_move(grid,"O",o_board,o_score)
        o_path=update_path(loc,o_path,o_move)
        o_move=o_move+1
        solved,direction=checkWinner(grid,"O",loc)
        if solved:
            print("O wins")
            if print_:
                print("loc= ")
                print(loc)
                print("direction= ")
                print(direction)
            x_board,x_score=first_update_learn(x_path,grid.copy(),"loser",x_board,x_score,learn_rate)
            o_board,o_score=first_update_learn(o_path,grid.copy(),"winner",o_board,o_score,learn_rate)
            return grid,x_board,x_score,o_board,o_score
    print("tie")
    x_board,x_score=first_update_learn(x_path,grid.copy(),"tie",x_board,x_score,learn_rate)
    o_board,o_score=first_update_learn(o_path,grid.copy(),"tie",o_board,o_score,learn_rate)
    return grid,x_board,x_score,o_board,o_score

In [17]:
def rand_move(grid,sign):
    #dummy move, used for setup testing
    i=0
    while(True):
        x=random.randint(0,3)
        y=random.randint(0,3)
        z=random.randint(0,3)
        if not grid[x,y,z]:
            grid[x,y,z]=sign
            return grid,[x,y,z]

In [18]:
def learn_move(grid,sign,board,score):
    ##
    ###dom place
    ##
    
    ##add if find a 3 in a row for itself use it
    ## add if it finds a opponets 3 in a row, get the place
    ## add if it finds a possible fork, with one move, make 3 in a row without any oppent blocking it
    ## add if it finds an oppenent possible fork, with one move the oppenent can make 3 in a row without you blocking it
    
    #if we are in new board: pick a random spot
    if len(np.where((board == grid).all(axis=1)))==0:
        return rand_move(grid,sign)
    else:
        
    #the moves that learns

    i=0
    while(True):
        x=random.randint(0,3)
        y=random.randint(0,3)
        z=random.randint(0,3)
        if not grid[x,y,z]:
            grid[x,y,z]=sign
            return grid,[x,y,z]

In [19]:
def checkWinner(grid,sign,loc):
    #goes through every direction in x,y,z directions
    spot=grid[loc[0],loc[1],loc[2]]
    for i in [0,1]:
        for j in [0,1]:
            for k in [0,1]:
                if(i==j==k==0):
                    next
                #some directions can have negative chooses so dealing with that
                else:
                    solved,direction=setDirection(grid,sign,loc,[i,j,k])
                    if solved:
                        return True, direction
    return False,0

In [20]:
#changes vector's part to negative if needed
def setDirection(grid,sign,loc,direction):
    #one direction, 
    if sum(direction)==1:
        return checkDirection(grid,sign,loc,direction)
    if sum(direction)==2:
        #trying to find the zero
        for i in range(0,3):
            if direction[i]==0:
                break
        # i is the location of the zero:
        # based on my studying:
        #even direction is when both dimesions are positive
        #odd direction is when one of the dimesions are negative
        #based on studies:
        ## if both the dimesions that have nonzero direction are the same then the:
            ###direction is positive
        ## if the dimesions that have nonzero direction have the sum of three 
            ###direction is negative
        new_loc=loc.copy()
        new_loc.pop(i)
        if new_loc[0]==new_loc[1]:
            return checkDirection(grid,sign,loc,direction)
        elif sum(new_loc)%4==3:
                if i==0:
                    direction[1]=direction[1]*-1
                else:
                    direction[0]=direction[0]*-1
                return checkDirection(grid,sign,loc,direction)
        #else, it does not work for that side
        return False,0
    else:
        #so the direction is based on where are the two equal values are located
            ##the direction of the equal values should be the same but the other dimension
        #direction is (1,1,1): values are all the same
        ###if they are all not equal then there is no 3d directional diagonal
        if len(set(loc))==1:
            return checkDirection(grid,sign,loc,direction)
        elif len(set(loc))==2:
            for i in range(len(direction)):
                new_loc=loc.copy()
                new_loc.pop(i)
                if len(set(new_loc))==1:
                    direction[i]=direction[i]*-1
                    break
            if (loc[i]+loc[(i+1)%3])%4==3:        
                return checkDirection(grid,sign,loc,direction)
            else:
                return False, 0
        else:
            return False,0
        

In [21]:
def checkDirection(grid,sign,loc,direction):
    #checks if the blocks in that direction have the same sign
    let =np.zeros(shape=(3),dtype=np.int)
    for i in range(0,4):
        for j in range(len(direction)):
            #used mod because I did not want to check the opposite direction
            let[j]=(direction[j]*(i)+loc[j])%4
        if grid[let[0],let[1],let[2]]!=sign:
            return False,0
    let =np.zeros(shape=(3),dtype=np.int)
    return True,direction

test

In [39]:
grid,x,o=setupGame(True,"rand")

X wins
loc= 
[3, 0, 0]
direction= 
[0, 1, 1]


In [40]:
x

array([[[ 0,  0,  0,  0],
        [ 2,  0,  0,  0],
        [ 0,  0,  0,  0],
        [10,  0,  0,  0]],

       [[ 0,  0,  5,  0],
        [ 0,  0,  7,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0]],

       [[ 0,  0,  0,  0],
        [ 0, 11,  6,  0],
        [ 3,  0,  0,  0],
        [12,  0,  0,  0]],

       [[13,  0,  0,  0],
        [ 0,  4,  0,  0],
        [ 0,  0,  8,  0],
        [ 1,  0,  0,  9]]], dtype=int32)

In [47]:
np.where(x == np.max(x))

(array([3]), array([0]), array([0]))

In [67]:
loc= [3, 1, 3]
direction= [1, -1, 1]

x=np.array([[['O W', '', '', 'O'],
        ['', '', '', ''],
        ['O', '', '', 'X'],
        ['', '', '', '']],

       [['X', '', '', ''],
        ['', '', 'X', ''],
        ['', '', '', ''],
        ['X', 'O W', '', '']],

       [['', '', '', ''],
        ['', '', '', ''],
        ['', '', 'O W', ''],
        ['', '', '', '']],

       [['', '', 'X', ''],
        ['', '', '', 'O S'],
        ['', '', '', ''],
        ['', '', 'X', '']]], dtype='<U25')



checkWinner(x,"X",loc)

(False, 0)

In [ ]:
## 1-d direction

In [ ]:
#columns

In [76]:
sol_grid=np.zeros(shape=(4,4,4),dtype=np.dtype('U25'))
for k in range(0,4):
    for l in range(0,4):
        sol_grid=np.zeros(shape=(4,4,4),dtype=np.dtype('U25'))
        for i in range(0,4):
            sol_grid[i,k,l]="X"
        for j in range(0,4):
            solved, direction=checkWinner(sol_grid,"X",[j,k,l])
            if not solved:
                print("Problem with columns")
                print(sol_grid)

In [ ]:
#rows

In [77]:
sol_grid=np.zeros(shape=(4,4,4),dtype=np.dtype('U25'))
for k in range(0,4):
    for l in range(0,4):
        sol_grid=np.zeros(shape=(4,4,4),dtype=np.dtype('U25'))
        for i in range(0,4):
            sol_grid[k,i,l]="X"
        for j in range(0,4):
            solved, direction=checkWinner(sol_grid,"X",[k,i,l])
            if not solved:
                print("Problem with rows")
                print(sol_grid)

In [ ]:
#verticals

In [78]:
sol_grid=np.zeros(shape=(4,4,4),dtype=np.dtype('U25'))
for k in range(0,4):
    for l in range(0,4):
        sol_grid=np.zeros(shape=(4,4,4),dtype=np.dtype('U25'))
        for i in range(0,4):
            sol_grid[l,k,i]="X"
        for j in range(0,4):
            solved, direction=checkWinner(sol_grid,"X",[l,k,j])
            if not solved:
                print("Problem with columns")
                print(sol_grid)

In [ ]:
#Done

In [ ]:
#2d

In [ ]:
#3d

In [140]:
sol_grid=np.zeros(shape=(4,4,4),dtype=np.dtype('U25'))
x=0
y=0
z=0
for i in range(0,4):
    sol_grid[x+i,y+i,z+i]="X"
for j in range(0,4):
    solved, direction=checkWinner(sol_grid,"X",[j,j,j])
    if not solved:
        print("Problem with columns")
        print(sol_grid)
sol_grid=np.zeros(shape=(4,4,4),dtype=np.dtype('U25'))

In [79]:
y_sol_grid=np.zeros(shape=(4,4,4),dtype=np.str)
for i in range(0,4):
    y_sol_grid[0,i,0]="X"

In [80]:
checkWinner(y_sol_grid,"X",[0,0,0])

True

In [83]:
z_sol_grid=np.zeros(shape=(4,4,4),dtype=np.str)
for i in range(0,4):
    z_sol_grid[0,i,0]="X"

In [84]:
checkWinner(z_sol_grid,"X",[0,0,0])

True